In [2]:
from azureml.core import Workspace, Experiment

subscription_id = '61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30'
resource_group = 'aml-quickstarts-139348'
workspace_name = 'quick-starts-ws-139348'

ws = Workspace(subscription_id, resource_group, workspace_name)
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-139348
Azure region: southcentralus
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-139348


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({
       "--C": uniform(0, 1)
   })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

from azureml.core import ScriptRunConfig

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', compute_target=cpu_cluster, entry_script='train.py')
# est = ScriptRunConfig(source_directory='./', compute_target=cpu_cluster, script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(# run_config=est,
                                estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(config = hyperdrive_config)
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [20]:
import joblib
# Get your best run and save the model from that run.


best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(best_run_metrics, parameter_values)

joblib.dump(best_run, 'outputs/model.joblib')
# model = best_run.register_model(model_name='project-model', model_path='outputs/model.joblib')

{'Regularization Strength:': 0.7226943673559209, 'Max iterations:': 100, 'Accuracy': 0.913814032532168} ['--C', '0.7226943673559209']


In [21]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [26]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x['target'] = y


from azureml.core.datastore import Datastore

datastore = Datastore.get(ws, 'workspacefilestore')
dataset = TabularDatasetFactory.register_pandas_dataframe(x, name = 'ProjectData', target = datastore)

In [32]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_settings = {
       "n_cross_validations": 3,
       "primary_metric": 'accuracy',
       "enable_early_stopping": True,
       "experiment_timeout_hours": 0.5,
       "max_concurrent_iterations": 4,
       "max_cores_per_iteration": -1,
       "verbosity": logging.INFO,
   }

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    training_data=dataset,
    label_column_name='target',
    compute_target = cpu_cluster,
    **automl_settings)

In [36]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)

Running on remote.
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_5bccd83f-da96-48b4-9ce6-ce260da1de90

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+------------

In [37]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = automl_run.get_output()

joblib.dump(fitted_model, 'outputs/model2.joblib')
# model = best_run.register_model(model_name='project-model', model_path='outputs/model.joblib')

Package:azureml-automl-runtime, training version:1.22.0, current version:1.20.0
Package:azureml-core, training version:1.22.0, current version:1.20.0
Package:azureml-dataprep, training version:2.9.1, current version:2.7.3
Package:azureml-dataprep-native, training version:29.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.7.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.22.0, current version:1.20.0
Package:azureml-defaults, training version:1.22.0, current version:1.20.0
Package:azureml-interpret, training version:1.22.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.22.0, current version:1.20.0
Package:azureml-telemetry, training version:1.22.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.22.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.22.0, current version:1.20.0


['outputs/model2.joblib']